## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-08-02-37-37 +0000,nypost,Rape victim horrified to learn her attacker fr...,https://nypost.com/2026/02/07/us-news/californ...
1,2026-02-08-02-35-25 +0000,nyt,"In Bid to Lead Thailand, a Progressive Party S...",https://www.nytimes.com/2026/02/07/world/asia/...
2,2026-02-08-02-32-21 +0000,latimes,A wolf has come to Los Angeles County for the ...,https://www.latimes.com/environment/story/2026...
3,2026-02-08-02-19-58 +0000,nypost,Gavin Newsom spotted getting cozy with sports ...,https://nypost.com/2026/02/07/us-news/gavin-ne...
4,2026-02-08-02-04-51 +0000,nypost,Heartless thief steals 9-year-old special need...,https://nypost.com/2026/02/07/us-news/heartles...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,21
48,new,13
36,bowl,10
116,mayor,9
35,super,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...,70
48,2026-02-07-23-41-35 +0000,nypost,Savannah Guthrie says she has received message...,https://nypost.com/2026/02/07/us-news/savannah...,66
5,2026-02-08-02-01-40 +0000,nyt,"Savannah Guthrie, in New Video Message, Promis...",https://www.nytimes.com/2026/02/07/us/savannah...,54
102,2026-02-07-16-52-10 +0000,nypost,Tehran’s foreign minister warns they’re ready ...,https://nypost.com/2026/02/07/world-news/tehra...,52
180,2026-02-07-03-03-00 +0000,wsj,The Trump administration announced a new strat...,https://www.wsj.com/politics/national-security...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,70,2026-02-07-03-07-00 +0000,wsj,A federal judge has ordered the Trump administ...,https://www.wsj.com/politics/policy/judge-bloc...
48,53,2026-02-07-23-41-35 +0000,nypost,Savannah Guthrie says she has received message...,https://nypost.com/2026/02/07/us-news/savannah...
112,36,2026-02-07-16-04-08 +0000,nypost,Lindsey Horvath won’t run for LA mayor after f...,https://nypost.com/2026/02/07/us-news/lindsey-...
78,35,2026-02-07-20-55-27 +0000,eltecolote,‘We’re not criminals’: Hot dog vendors say S.F...,https://eltecolote.org/content/en/sf-food-vend...
77,31,2026-02-07-20-56-48 +0000,nypost,Epstein cultivated close ties with two key Rus...,https://nypost.com/2026/02/07/world-news/epste...
57,31,2026-02-07-23-09-26 +0000,startribune,The latest: Jessie Diggins finishes eighth in ...,https://www.startribune.com/2026-winter-olympi...
161,31,2026-02-07-09-00-00 +0000,cbc,Should you stay or should you go? Canadians re...,https://www.cbc.ca/news/world/cuba-travel-advi...
14,28,2026-02-08-01-47-09 +0000,nypost,Meet Ivy League activist and socialist union b...,https://nypost.com/2026/02/07/us-news/meet-ca-...
49,28,2026-02-07-23-39-50 +0000,nypost,Washington Post publisher Will Lewis steps dow...,https://nypost.com/2026/02/07/us-news/washingt...
69,28,2026-02-07-21-38-40 +0000,nypost,Trump says US held ‘very good talks with Iran’...,https://nypost.com/2026/02/07/world-news/trump...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
